In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (222, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


# 2. Load Center Coords by Candidates

In [4]:
lungs = list(set(df_meta_test.index))
print('distinct lungs in testset:', len(lungs))

distinct lungs in testset: 222


In [5]:
df_tasks = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'candidates/tasks_medi.csv', 
                            index_col=['seriesuid'])
df_tasks.index = df_tasks.index.astype('str')

print('test tasks:', df_tasks.shape, 'distinct lung:', len(set(df_tasks.index)))

test tasks: (38875, 6) distinct lung: 177


In [6]:
# def nms(records, thresh):
#     x1 = records[:, 'vcoordX'] - records[:, 'diameterX'] / 2
#     y1 = records[:, 'vcoordY'] - records[:, 'diameterY'] / 2
#     x2 = records[:, 'vcoordX'] + records[:, 'diameterX'] / 2
#     y2 = records[:, 'vcoordY'] + records[:, 'diameterY'] / 2
#     scores = record[:, 'probability']

#     areas = (x2 - x1 + 1) * (y2 - y1 + 1)
#     order = scores.argsort()[::-1]

#     keep = []
#     while order.size > 0:
#         i = order[0]
#         keep.append(i)
#         xx1 = np.maximum(x1[i], x1[order[1:]])
#         yy1 = np.maximum(y1[i], y1[order[1:]])
#         xx2 = np.minimum(x2[i], x2[order[1:]])
#         yy2 = np.minimum(y2[i], y2[order[1:]])

#         w = np.maximum(0.0, xx2 - xx1 + 1)
#         h = np.maximum(0.0, yy2 - yy1 + 1)
#         inter = w * h
#         ovr = inter / (areas[i] + areas[order[1:]] - inter)

#         inds = np.where(ovr <= thresh)[0]
#         order = order[inds + 1]

#     return keep

# list_tasks_nms = []
# for uid in set(df_test_tasks.index):
#     tasks = df_test_tasks.loc[[uid]] # DataFrame
#     list_tasks_nms.extend(nms(tasks, 0.8))

# 3. Predict Results by Task_test.csv

In [7]:
df_tasks_nms = df_tasks

# df_tasks_nms = pd.DataFrame(list_tasks_nms, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
# df_tasks_nms.index = df_tasks_nms.index.astype('str')

print('test tasks:', df_tasks_nms.shape, 'distinct lung:', len(set(df_tasks_nms.index)))

test tasks: (38875, 6) distinct lung: 177


In [8]:
df_tasks_nms.head(10)

,vcoordX,vcoordY,vcoordZ,diametersX,diametersY,diametersZ
seriesuid,,,,,,
680203,287,30,29,15,14,16
680203,46,68,27,24,29,28
680203,310,72,26,14,18,13
680203,42,97,40,24,34,31
680203,40,130,23,30,48,34
680203,319,129,25,10,17,14
680203,33,153,31,5,6,7
680203,202,198,21,21,24,18
680203,189,221,26,11,8,10


In [9]:
import tensorflow.keras.backend as K
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [10]:
from tensorflow.keras.models import load_model
output_dir = './output/densenet3d/1564138383/' # labels in center coord
model = load_model(output_dir + "model-best.hd5")

W0729 13:51:23.023361 140359731586880 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 13:51:23.024223 140359731586880 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 13:51:23.030501 140359731586880 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [11]:
labels = [0, 31, 32]
batch_size = 128
threshold_probability = 0.7
wtype = 'medi'
plot = False

In [12]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [ ]:
list_results = []
print(wtype, 'window')
count_lung = 0
for uid in set(df_tasks_nms.index):
    tasks = df_tasks_nms.loc[[uid]] # DataFrame
    meta = df_meta_test.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks))
#     print(meta)
                 

    lung_m, mask_m = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='test', wtype='medi')

    lung = (lung_m*(mask_m>0)).astype('float') / 255

    # convert coordX to vcoordX
#     tasks['vcoordX'] = tasks['coordX'] - meta.originX
#     tasks['vcoordY'] = tasks['coordY'] - meta.originY
#     tasks['vcoordZ'] = tasks['coordZ'] - meta.originZ

    count = 0
    for idx, item in tasks.iterrows():     
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []
            
        flag_range = True
        if (item.vcoordX < 100) or (item.vcoordX > 250):
            flag_range = False
        if (item.vcoordY < 50) or (item.vcoordY > 250):
            flag_range = False
        if item.vcoordZ > 350:
            flag_range = False

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=32)

        if np.sum(cube) > 0 and flag_range:
            X_item.append(item)
            X_test.append(cube)
        # else:
            # print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            predictions_test = model.predict(X_test, batch_size=batch_size*5, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                if np.argmax(predictions_test[i]) > 0:
                    category = int(np.argmax(predictions_test[i]))
                    probability = float(np.max(predictions_test[i]))

                    result = {}
                    result['seriesuid'] = uid
                    result['coordX'] = itm.vcoordX + meta.originX
                    result['coordY'] = itm.vcoordY + meta.originY
                    result['coordZ'] = itm.vcoordZ + meta.originZ
                    result['class'] = helper.get_origin_class_by_predicted_value(category, labels)
                    result['probability'] = probability
                    result['wtype'] = wtype

                    if probability > threshold_probability:
                        print(result)
                        if plot:
                            plot_cube((cube*255).astype(np.uint8), rows=4, cols=8)
                        list_results.append(result)

        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','coordX','coordY','coordZ','class','probability','wtype'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')
    
    count_lung += 1
    print('total:', df_results.shape, 'lung:', len(set(df_results.index)), '/', count_lung)

    df_results.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/submission_' + wtype +'.csv', encoding='utf-8')

    print('-'*100)
    

medi window
lung: 672198 tasks: 88
36/36 [==============================] - 3s 94ms/sample
{'seriesuid': '672198', 'coordX': -38.18799999999999, 'coordY': -66.18799999999999, 'coordZ': -553.4, 'class': 31, 'probability': 0.9999715089797974, 'wtype': 'medi'}
{'seriesuid': '672198', 'coordX': -0.18799999999998818, 'coordY': -213.188, 'coordZ': -517.4, 'class': 31, 'probability': 0.9995235204696655, 'wtype': 'medi'}
{'seriesuid': '672198', 'coordX': 33.81200000000001, 'coordY': -210.188, 'coordZ': -528.4, 'class': 31, 'probability': 0.9744600057601929, 'wtype': 'medi'}
{'seriesuid': '672198', 'coordX': -59.18799999999999, 'coordY': -57.18799999999999, 'coordZ': -539.4, 'class': 31, 'probability': 0.9999074935913086, 'wtype': 'medi'}
{'seriesuid': '672198', 'coordX': 34.81200000000001, 'coordY': -54.18799999999999, 'coordZ': -529.4, 'class': 32, 'probability': 0.8806232810020447, 'wtype': 'medi'}
{'seriesuid': '672198', 'coordX': 2.812000000000012, 'coordY': -210.188, 'coordZ': -490.4, 'cl

25/25 [==============================] - 0s 4ms/sample
{'seriesuid': '631173', 'coordX': -7.813999999999993, 'coordY': -106.65799999999999, 'coordZ': -1104.0, 'class': 31, 'probability': 0.9941093921661377, 'wtype': 'medi'}
{'seriesuid': '631173', 'coordX': 10.186000000000007, 'coordY': -106.65799999999999, 'coordZ': -1099.0, 'class': 31, 'probability': 0.8105989694595337, 'wtype': 'medi'}
{'seriesuid': '631173', 'coordX': 42.18600000000001, 'coordY': -112.65799999999999, 'coordZ': -1113.0, 'class': 32, 'probability': 0.9673261642456055, 'wtype': 'medi'}
{'seriesuid': '631173', 'coordX': 62.18600000000001, 'coordY': -104.65799999999999, 'coordZ': -1103.0, 'class': 32, 'probability': 0.9744980931282043, 'wtype': 'medi'}
{'seriesuid': '631173', 'coordX': 39.18600000000001, 'coordY': 75.34200000000001, 'coordZ': -1100.0, 'class': 31, 'probability': 0.9905675053596497, 'wtype': 'medi'}
{'seriesuid': '631173', 'coordX': 35.18600000000001, 'coordY': -106.65799999999999, 'coordZ': -1083.0, 'c

57/57 [==============================] - 0s 3ms/sample
{'seriesuid': '530142', 'coordX': -17.0, 'coordY': 14.0, 'coordZ': -201.0, 'class': 32, 'probability': 0.9250339269638062, 'wtype': 'medi'}
{'seriesuid': '530142', 'coordX': -55.0, 'coordY': 62.0, 'coordZ': -194.0, 'class': 31, 'probability': 0.9875125288963318, 'wtype': 'medi'}
{'seriesuid': '530142', 'coordX': 2.0, 'coordY': 55.0, 'coordZ': -200.0, 'class': 32, 'probability': 0.9915128350257874, 'wtype': 'medi'}
{'seriesuid': '530142', 'coordX': -81.0, 'coordY': 66.0, 'coordZ': -181.0, 'class': 32, 'probability': 0.9983421564102173, 'wtype': 'medi'}
{'seriesuid': '530142', 'coordX': -58.0, 'coordY': 63.0, 'coordZ': -195.0, 'class': 31, 'probability': 0.8836362361907959, 'wtype': 'medi'}
{'seriesuid': '530142', 'coordX': -22.0, 'coordY': 51.0, 'coordZ': -160.0, 'class': 31, 'probability': 0.9992020726203918, 'wtype': 'medi'}
{'seriesuid': '530142', 'coordX': -15.0, 'coordY': -92.0, 'coordZ': -125.0, 'class': 32, 'probability': 0.9

131/131 [==============================] - 0s 3ms/sample
{'seriesuid': '526336', 'coordX': 85.6, 'coordY': -54.5, 'coordZ': -873.59, 'class': 31, 'probability': 0.9929426908493042, 'wtype': 'medi'}
{'seriesuid': '526336', 'coordX': 73.6, 'coordY': 2.5, 'coordZ': -867.59, 'class': 31, 'probability': 0.9999194145202637, 'wtype': 'medi'}
{'seriesuid': '526336', 'coordX': 10.599999999999994, 'coordY': 76.5, 'coordZ': -868.59, 'class': 31, 'probability': 0.9990567564964294, 'wtype': 'medi'}
{'seriesuid': '526336', 'coordX': 50.599999999999994, 'coordY': 84.5, 'coordZ': -875.59, 'class': 31, 'probability': 0.9999719858169556, 'wtype': 'medi'}
{'seriesuid': '526336', 'coordX': 36.599999999999994, 'coordY': 81.5, 'coordZ': -863.59, 'class': 31, 'probability': 0.9999067783355713, 'wtype': 'medi'}
{'seriesuid': '526336', 'coordX': 85.6, 'coordY': 79.5, 'coordZ': -870.59, 'class': 31, 'probability': 0.8120896816253662, 'wtype': 'medi'}
{'seriesuid': '526336', 'coordX': 11.599999999999994, 'coordY

28/28 [==============================] - 0s 4ms/sample
{'seriesuid': '689170', 'coordX': 95.18200000000002, 'coordY': -16.712999999999994, 'coordZ': 410.543, 'class': 31, 'probability': 0.9999997615814209, 'wtype': 'medi'}
{'seriesuid': '689170', 'coordX': -8.817999999999984, 'coordY': 52.287000000000006, 'coordZ': 406.543, 'class': 31, 'probability': 0.9968249797821045, 'wtype': 'medi'}
{'seriesuid': '689170', 'coordX': 27.182000000000016, 'coordY': 52.287000000000006, 'coordZ': 408.543, 'class': 31, 'probability': 0.892494261264801, 'wtype': 'medi'}
{'seriesuid': '689170', 'coordX': -35.817999999999984, 'coordY': -19.712999999999994, 'coordZ': 447.543, 'class': 31, 'probability': 0.9985378980636597, 'wtype': 'medi'}
{'seriesuid': '689170', 'coordX': -13.817999999999984, 'coordY': -23.712999999999994, 'coordZ': 444.543, 'class': 31, 'probability': 0.999782145023346, 'wtype': 'medi'}
{'seriesuid': '689170', 'coordX': 27.182000000000016, 'coordY': 5.287000000000006, 'coordZ': 482.543, '

...

41/41 [==============================] - 0s 3ms/sample
{'seriesuid': '672337', 'coordX': 57.34200000000001, 'coordY': -55.65799999999999, 'coordZ': -285.034, 'class': 32, 'probability': 0.7130731344223022, 'wtype': 'medi'}
{'seriesuid': '672337', 'coordX': -21.657999999999987, 'coordY': 66.34200000000001, 'coordZ': -293.034, 'class': 31, 'probability': 0.9926983714103699, 'wtype': 'medi'}
{'seriesuid': '672337', 'coordX': -23.657999999999987, 'coordY': 58.34200000000001, 'coordZ': -290.034, 'class': 31, 'probability': 0.9803697466850281, 'wtype': 'medi'}
{'seriesuid': '672337', 'coordX': -22.657999999999987, 'coordY': 46.34200000000001, 'coordZ': -281.034, 'class': 31, 'probability': 0.994620680809021, 'wtype': 'medi'}
{'seriesuid': '672337', 'coordX': -24.657999999999987, 'coordY': 58.34200000000001, 'coordZ': -261.034, 'class': 31, 'probability': 0.9907611012458801, 'wtype': 'medi'}
{'seriesuid': '672337', 'coordX': 20.342000000000013, 'coordY': 47.34200000000001, 'coordZ': -256.034,

94/94 [==============================] - 0s 3ms/sample
{'seriesuid': '662265', 'coordX': -77.69999999999999, 'coordY': -112.0, 'coordZ': -958.3, 'class': 32, 'probability': 0.8129312992095947, 'wtype': 'medi'}
{'seriesuid': '662265', 'coordX': 49.30000000000001, 'coordY': -112.0, 'coordZ': -965.3, 'class': 31, 'probability': 0.9994868040084839, 'wtype': 'medi'}
{'seriesuid': '662265', 'coordX': -43.69999999999999, 'coordY': -19.0, 'coordZ': -955.3, 'class': 31, 'probability': 0.7757736444473267, 'wtype': 'medi'}
{'seriesuid': '662265', 'coordX': -50.69999999999999, 'coordY': 56.0, 'coordZ': -956.3, 'class': 31, 'probability': 0.9989815354347229, 'wtype': 'medi'}
{'seriesuid': '662265', 'coordX': 45.30000000000001, 'coordY': 57.0, 'coordZ': -957.3, 'class': 31, 'probability': 0.8988715410232544, 'wtype': 'medi'}
{'seriesuid': '662265', 'coordX': -56.69999999999999, 'coordY': -119.0, 'coordZ': -926.3, 'class': 31, 'probability': 0.9899218678474426, 'wtype': 'medi'}
{'seriesuid': '662265'

24/24 [==============================] - 0s 3ms/sample
{'seriesuid': '696568', 'coordX': -30.671999999999997, 'coordY': -209.67200000000003, 'coordZ': -100.19999999999999, 'class': 31, 'probability': 1.0, 'wtype': 'medi'}
{'seriesuid': '696568', 'coordX': 14.328000000000003, 'coordY': -208.67200000000003, 'coordZ': -72.19999999999999, 'class': 31, 'probability': 0.815657913684845, 'wtype': 'medi'}
{'seriesuid': '696568', 'coordX': -20.671999999999997, 'coordY': -172.67200000000003, 'coordZ': -61.19999999999999, 'class': 31, 'probability': 0.9586508274078369, 'wtype': 'medi'}
{'seriesuid': '696568', 'coordX': 75.328, 'coordY': -59.672000000000025, 'coordZ': -64.19999999999999, 'class': 31, 'probability': 0.8603312969207764, 'wtype': 'medi'}
{'seriesuid': '696568', 'coordX': -48.672, 'coordY': -206.67200000000003, 'coordZ': -42.19999999999999, 'class': 32, 'probability': 0.8038923740386963, 'wtype': 'medi'}
{'seriesuid': '696568', 'coordX': 33.328, 'coordY': -216.67200000000003, 'coordZ'

38/38 [==============================] - 0s 3ms/sample
{'seriesuid': '655450', 'coordX': 67.25799999999998, 'coordY': -73.65799999999999, 'coordZ': -87.63499999999999, 'class': 32, 'probability': 0.723009467124939, 'wtype': 'medi'}
{'seriesuid': '655450', 'coordX': -16.74200000000002, 'coordY': 22.342000000000013, 'coordZ': -96.63499999999999, 'class': 32, 'probability': 0.9270574450492859, 'wtype': 'medi'}
{'seriesuid': '655450', 'coordX': -27.74200000000002, 'coordY': -86.65799999999999, 'coordZ': -74.63499999999999, 'class': 32, 'probability': 0.9978635907173157, 'wtype': 'medi'}
{'seriesuid': '655450', 'coordX': -0.7420000000000186, 'coordY': -76.65799999999999, 'coordZ': -70.63499999999999, 'class': 32, 'probability': 0.9999649524688721, 'wtype': 'medi'}
{'seriesuid': '655450', 'coordX': 68.25799999999998, 'coordY': -72.65799999999999, 'coordZ': -55.63499999999999, 'class': 32, 'probability': 0.9833005666732788, 'wtype': 'medi'}
{'seriesuid': '655450', 'coordX': 2.2579999999999814

47/47 [==============================] - 0s 3ms/sample
{'seriesuid': '656748', 'coordX': 9.864000000000004, 'coordY': -231.13599999999997, 'coordZ': -344.8, 'class': 32, 'probability': 0.9862236380577087, 'wtype': 'medi'}
{'seriesuid': '656748', 'coordX': -82.136, 'coordY': -233.13599999999997, 'coordZ': -314.8, 'class': 32, 'probability': 0.9999555349349976, 'wtype': 'medi'}
{'seriesuid': '656748', 'coordX': -79.136, 'coordY': -232.13599999999997, 'coordZ': -314.8, 'class': 32, 'probability': 0.9997856020927429, 'wtype': 'medi'}
{'seriesuid': '656748', 'coordX': -64.136, 'coordY': -234.13599999999997, 'coordZ': -318.8, 'class': 32, 'probability': 0.9993075132369995, 'wtype': 'medi'}
{'seriesuid': '656748', 'coordX': -74.136, 'coordY': -123.13599999999997, 'coordZ': -302.8, 'class': 32, 'probability': 0.9989590644836426, 'wtype': 'medi'}
{'seriesuid': '656748', 'coordX': 17.864000000000004, 'coordY': -73.13599999999997, 'coordZ': -304.8, 'class': 31, 'probability': 0.9419516921043396, 

In [ ]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'candidates' + '/submission_' + wtype +'.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))

In [ ]:
df_results = df_results[df_results['probability'] > 0.70]
df_results = df_results.drop(columns=['wtype'])
print(df_results.shape, 'distinct lung:', len(set(df_results.index)))

In [ ]:
df_results.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/submission_' + wtype + '_filtered.csv', encoding='utf-8')

print('31:', len(df_results[df_results['class']==31]), '32:', len(df_results[df_results['class']==32]))
print('1:', len(df_results[df_results['class']==1]), '5:', len(df_results[df_results['class']==5]))